In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import scipy.stats as sc
from sklearn import neighbors
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn import model_selection

In [2]:
def predictSales(df,model,test,stores,months):
    for store in stores:
        temp = test.loc[test['StoreID'] == store]
        for month in months:
            X = temp.loc[temp[month] == 1]
            X = X.drop(labels='StoreID', axis=1)
            res = model.predict(X)
            res = np.expm1(res)
            res = {'StoreID' : [store], 'Month' : [month], 'NumberOfSales' : [np.sum(res)]}
            res = pd.DataFrame(res)
            df = df.append(res,ignore_index=True)
    
    df.to_csv("Results.csv")

In [3]:

# Need train set, index of varibales, index of target, test size and seed

def randomSplit(dataset, variable,target,test_size=0.33, random_state=1234):

    train_data, test_data = model_selection.train_test_split(dataset, test_size=0.33, random_state=1234)


    train_x = train_data[variable].values
    train_y = train_data[target].values

    test_x = test_data[variable].values
    test_y = test_data[target].values

#    train_x = train_x.reshape(len(train_x), degree)
    train_y = train_y.reshape(len(train_y), 1)
#    test_x = test_x.reshape(len(test_y), degree)
    test_y = test_y.reshape(len(test_y), 1)


    return train_x,train_y,test_x,test_y

In [4]:
#stores = test['StoreID'].unique()
#months = ['March','April']
#df = pd.DataFrame(columns=['StoreID', 'Month', 'NumberOfSales'])
#predictSales(df,knn,test,stores,months)

# K Nearest Neighbors 

In [46]:
train = pd.read_csv('TrainClean.csv',index_col=0)
test = pd.read_csv('TestClean.csv',index_col=0)

In [47]:
target = train['NumberOfSales']
variables = train.drop(labels='NumberOfSales',axis=1)

In [48]:
train['NumberOfSales'].describe()

count    433958.000000
mean          2.239679
std           0.050325
min           0.000000
25%           2.211314
50%           2.241269
75%           2.270551
max           2.415042
Name: NumberOfSales, dtype: float64

In [49]:
#target = np.expm1(target.values)

In [50]:
#sc.describe(target)

In [51]:
var = train.columns.drop('NumberOfSales')
tar = 'NumberOfSales'

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(df[numerical_variables])
df_normalized_values = scaler.transform(df[numerical_variables])
df_normalized = pd.DataFrame(data = df_normalized_values, columns=numerical_variables)

In [52]:
RMSE = {}
R2 = {}

for ii in range(2,5):
    
    tr_x,tr_y,tes_x,tes_y = randomSplit(train,var,tar)
    
    n_neighbors = ii
    knn = neighbors.KNeighborsRegressor(n_neighbors)
    knn = knn.fit(tr_x, tr_y)

    yp = knn.predict(tes_x)
    yt = tes_y

    RMSE[ii]=np.sqrt (sum((yt - yp)*(yt - yp)) / len(yt))
    R2[ii]=knn.score(tes_x,tes_y)

    print("TEST RESULTS ->RMSE : "+str(RMSE[ii])+" R2 :"+str(R2[ii]) +" for KNN with "+str(n_neighbors)+" neighbors")

TEST RESULTS ->RMSE : [0.04291793] R2 :0.21128278961651506 for KNN with 2 neighbors
TEST RESULTS ->RMSE : [0.03966811] R2 :0.3262065645045549 for KNN with 3 neighbors
TEST RESULTS ->RMSE : [0.03804062] R2 :0.3803606164800103 for KNN with 4 neighbors


In [10]:
#scores = cross_val_score(knn, variables, target, cv=KFold(n_splits=10, random_state=1234, shuffle=True))

# Random Forset

In [1]:
from sklearn.ensemble import RandomForestRegressor
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import scipy.stats as sc
from sklearn import neighbors
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn import model_selection

In [2]:
train = pd.read_csv('TrainClean.csv',index_col=0)
test = pd.read_csv('TestClean.csv',index_col=0)

In [3]:
target = train['NumberOfSales']
variables = train.drop(labels='NumberOfSales',axis=1)

In [4]:
ensemble_clfs = [
    ("RandomForestRegressor, max_features='sqrt'",
        RandomForestRegressor(oob_score=True,
                               max_features="sqrt",
                               random_state=1234)),
    ("RandomForestRegressor, max_features='log2'",
        RandomForestRegressor(max_features='log2', oob_score=True,
                               random_state=1234)),
    ("RandomForestRegressor, max_features=None",
        RandomForestRegressor(max_features=None, oob_score=True,
                               random_state=1234))
]

In [5]:
estimators = 30
OOB_Scores = {}

for label, clf in ensemble_clfs:
        clf.set_params(n_estimators=estimators)
        clf.fit(variables, target)
        OOB_Scores[(label,clf)] = oob_error = 1 - clf.oob_score_

In [12]:
OOB_Scores

{("RandomForestRegressor, max_features='log2'",
  RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
             max_features='log2', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=30, n_jobs=1,
             oob_score=True, random_state=1234, verbose=0, warm_start=False)): 0.5324170249662762,
 ("RandomForestRegressor, max_features='sqrt'",
  RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
             max_features='sqrt', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=30, n_jobs=1,
             oob_score=True, random_state=1234, verbose=0, warm_start=False)): 0.45807719553335957,
 ('RandomForestRegressor, max_features=None',
  RandomForestRegressor(b

In [14]:
rfr = RandomForestRegressor(max_features='log2', oob_score=True,random_state=1234)
rfr.set_params(n_estimators=40)
rfr.fit(variables, target)
oob_error = 1 - rfr.oob_score_

In [15]:
oob_error

0.5201968734012892

In [35]:
rfr = RandomForestRegressor(max_features='log2', oob_score=True,random_state=1234,warm_start=True)
rfr.set_params(n_estimators=30)
rfr.fit(variables, target)
oob_error = 1 - rfr.oob_score_

In [36]:
oob_error

0.5324170249662762

In [41]:
oob_error = 1 - rfr.oob_score_

In [42]:
oob_error

0.5324170249662762

In [61]:
rfr = RandomForestRegressor(max_features='log2', min_samples_leaf=2000,oob_score=True,random_state=1234) #keep increase min_samples
rfr.set_params(n_estimators=30)
rfr.fit(variables, target)
oob_error = 1 - rfr.oob_score_

In [62]:
oob_error

0.7709708403455853

# Support Vector Machine

In [63]:
from sklearn.svm import SVR
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import scipy.stats as sc
from sklearn import neighbors
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn import model_selection

In [64]:
train = pd.read_csv('TrainClean.csv',index_col=0)
test = pd.read_csv('TestClean.csv',index_col=0)

In [ ]:
target = train['NumberOfSales']
variables = train.drop(labels='NumberOfSales',axis=1)

In [ ]:
clf = SVR(C=1.0, epsilon=0.2)
clf.fit(variables,target)
clf.score(variables,target)